# 2. Продвинутые операции

## 2.1 Joins и оконные функции

## 2.2 Работа с датами и строками

## 2.3 Оптимизация (партиционирование, кэширование)

## 2.4 UDF (пользовательские функции)

## 2.5 Чтение/запись в разных форматах